In [2]:
import os
import sys
sys.path.append("../")

import pandas as pd
import numpy as np
import datetime as dt
import re
import altair as alt

from src.utils.parser import parse_config
from src.utils.vault import get_secrets
from src.utils.processing import downcast
import snowflake.connector

config = parse_config(os.path.abspath(os.path.join(os.getcwd(), "../src/settings.yml")))

In [3]:
data_config = get_secrets("snowflake")
data_config.update(config["snowflake"]["data"])
meta_config = config["snowflake"]["meta_box"]
snowflake_ctx = snowflake.connector.connect(**data_config)

In [4]:
%%time
vestiging = "Breda"

query = f"""
SELECT t1.BOXID AS boxid
FROM {meta_config["database"]}.{meta_config["schema"]}.{meta_config["table"]} t1
WHERE t1.VESTIGING LIKE '{vestiging}'
"""

df_box = pd.read_sql(sql=query, con=snowflake_ctx)

CPU times: user 13.2 ms, sys: 10.1 ms, total: 23.3 ms
Wall time: 2.46 s


In [ ]:
%%time
ids_breda = df_box.squeeze().to_list()

query = f"""
SELECT
    t0.BOXID AS boxid,
--    t0.CHANNELID AS channelid,
    YEAROFWEEKISO(t0.DATUMTIJD) AS year,
    WEEKISO(t0.DATUMTIJD) AS week,
    COUNT(DISTINCT t0.DATUMTIJD)/(4*24*7) AS COMPLETENESS
FROM {data_config["database"]}.{data_config["schema"]}.{data_config["table"]} t0
WHERE t0.CHANNELID LIKE 'register://electricity/0/activepower/sumli?avg=15'
  AND t0.DATUMTIJD < DATEADD(DAY, -DAYOFWEEKISO(CURRENT_DATE), CURRENT_DATE)
  AND t0.BOXID IN ({",".join([f"'{id}'" for id in ids_breda])})
GROUP BY t0.BOXID, t0.CHANNELID, YEAROFWEEKISO(t0.DATUMTIJD), WEEKISO(t0.DATUMTIJD)
"""

df_query = (
    pd.read_sql(sql=query, con=snowflake_ctx)
    .apply(downcast, try_numeric=True, category=True)
#     .sort_values(["YEAR", "WEEK"])
)
df_query

In [ ]:
df = df_query.copy()
df["DATE"] =  df.apply(lambda d: dt.datetime.fromisocalendar(d["YEAR"], d["WEEK"], 1), axis=1)
df["ISO"] = df.apply(lambda d: f'{d["YEAR"]} - {d["WEEK"]:02}', axis=1)
df = df.sort_values(["DATE", "COMPLETENESS"], ascending=True)

df_index = df.drop_duplicates(subset="BOXID")["BOXID"].reset_index(drop=True).reset_index()
df = pd.merge(df, df_index, on="BOXID")

df

In [ ]:
alt.data_transformers.disable_max_rows()
(
    alt.Chart(df)
    .mark_rect()
    .encode(
        alt.X("ISO:O", title="year - week"),
        alt.Y("index:O", sort='descending', title=f"# DALI boxes"),
        alt.Color("COMPLETENESS:Q", scale=alt.Scale(scheme="greenblue"), title="completeness"),
        alt.Tooltip("BOXID")
    )
    .properties(
        width=800,
        height=500,
    )
    .interactive()
)

In [43]:
%%time
ids_breda = df_box.squeeze().to_list()

query = f"""
SELECT
    DATE(t0.DATUMTIJD) AS "DATE",
    COUNT(t0.DATUMTIJD) AS RECORDS
FROM {data_config["database"]}.{data_config["schema"]}.{data_config["table"]} t0
WHERE t0.CHANNELID LIKE 'register://electricity/0/activepower/sumli?avg=15'
  AND t0.DATUMTIJD < DATEADD(DAY, -DAYOFWEEKISO(CURRENT_DATE), CURRENT_DATE)
  AND t0.BOXID IN ({",".join([f"'{id}'" for id in ids_breda])})
GROUP BY DATE(t0.DATUMTIJD)
"""

df_count = (
    pd.read_sql(sql=query, con=snowflake_ctx)
)
df_count["DATE"] = pd.to_datetime(df_count["DATE"])
df_count

CPU times: user 237 ms, sys: 95.9 ms, total: 333 ms
Wall time: 3min 55s


,DATE,RECORDS
0,2021-05-11,36506
1,2021-03-09,34972
2,2020-12-02,34265
3,2020-09-28,33285
4,2020-09-18,33009
...,...,...
980,2018-09-13,3700
981,2018-09-09,3360
982,2019-02-21,15773
983,2018-10-27,6720


In [44]:
(
    alt.Chart(df_count)
    .mark_bar()
    .encode(
        alt.X("DATE:T", title="date"),
        alt.Y("RECORDS:Q", title=f"# counts of P sumli"),
        alt.Tooltip(("DATE", "RECORDS")),
    )
    .properties(width=900)
)

alt.Chart(...)

In [ ]:
# alternative (big and slow)
# %%time
# data_config = config["snowflake"]["data"]

# query = f"""
# SELECT 
#     t1.*
# FROM
#     (
#     SELECT
#         t0.BOXID AS boxid,
#     --    t0.CHANNELID AS channelid,
#         YEAROFWEEKISO(t0.DATUMTIJD) AS year,
#         WEEKISO(t0.DATUMTIJD) AS week,
#         COUNT(DISTINCT t0.DATUMTIJD)/(4*24*7) AS COMPLETENESS
#     FROM {data_config["database"]}.{data_config["schema"]}.{data_config["table"]} t0
#     WHERE t0.CHANNELID LIKE 'register://electricity/0/activepower/sumli?avg=15'
#       AND t0.DATUMTIJD < DATEADD(DAY, -DAYOFWEEKISO(CURRENT_DATE), CURRENT_DATE)
#     GROUP BY t0.BOXID, t0.CHANNELID, YEAROFWEEKISO(t0.DATUMTIJD), WEEKISO(t0.DATUMTIJD)
#     ) t1
# INNER JOIN
# (
#     SELECT t2.BOXID AS boxid
#     FROM {meta_config["database"]}.{meta_config["schema"]}.{meta_config["table"]} t2
#     WHERE t2.VESTIGING LIKE '{vestiging}'
# ) t3
# ON t1.BOXID = t3.BOXID
# """

# df_query = (
#     pd.read_sql(sql=query, con=snowflake_ctx)
#     .apply(downcast, try_numeric=True, category=True)
#     .sort_values(["YEAR", "WEEK"])
# )
# df_query